In [14]:
import holidays
import numpy as np

In [19]:
holidays.US(language="en_US", years=np.arange(1995, 2045))

{datetime.date(1995, 1, 1): "New Year's Day", datetime.date(1995, 1, 2): "New Year's Day (observed)", datetime.date(1995, 5, 29): 'Memorial Day', datetime.date(1995, 7, 4): 'Independence Day', datetime.date(1995, 9, 4): 'Labor Day', datetime.date(1995, 11, 11): 'Veterans Day', datetime.date(1995, 11, 10): 'Veterans Day (observed)', datetime.date(1995, 11, 23): 'Thanksgiving', datetime.date(1995, 12, 25): 'Christmas Day', datetime.date(1995, 1, 16): 'Martin Luther King Jr. Day', datetime.date(1995, 2, 20): "Washington's Birthday", datetime.date(1995, 10, 9): 'Columbus Day', datetime.date(1996, 1, 1): "New Year's Day", datetime.date(1996, 5, 27): 'Memorial Day', datetime.date(1996, 7, 4): 'Independence Day', datetime.date(1996, 9, 2): 'Labor Day', datetime.date(1996, 11, 11): 'Veterans Day', datetime.date(1996, 11, 28): 'Thanksgiving', datetime.date(1996, 12, 25): 'Christmas Day', datetime.date(1996, 1, 15): 'Martin Luther King Jr. Day', datetime.date(1996, 2, 19): "Washington's Birthday

In [23]:
# -*- coding: utf-8 -*-
# Copyright (c) Facebook, Inc. and its affiliates.

# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

from __future__ import absolute_import, division, print_function

import numpy as np
import pandas as pd

import holidays


def get_country_holidays_class(country):
    """Get class for a supported country.

    Parameters
    ----------
    country: country code

    Returns
    -------
    A valid country holidays class
    """
    substitutions = {
        "TU": "TR",  # For compatibility with Turkey as 'TU' cases.
    }

    country = substitutions.get(country, country)
    if not hasattr(holidays, country):
        raise AttributeError(f"Holidays in {country} are not currently supported!")

    return getattr(holidays, country)


def get_holiday_names(country):
    """Return all possible holiday names of given country

    Parameters
    ----------
    country: country name

    Returns
    -------
    A set of all possible holiday names of given country
    """
    country_holidays = get_country_holidays_class(country)
    return set(country_holidays(language="en_US", years=np.arange(1995, 2045)).values())


def make_holidays_df(year_list, country, province=None, state=None):
    """Make dataframe of holidays for given years and countries

    Parameters
    ----------
    year_list: a list of years
    country: country name

    Returns
    -------
    Dataframe with 'ds' and 'holiday', which can directly feed
    to 'holidays' params in Prophet
    """
    country_holidays = get_country_holidays_class(country)
    holidays = country_holidays(expand=False, language="en_US", subdiv=province, years=year_list)

    holidays_df = pd.DataFrame(
        [(date, holidays.get_list(date)) for date in holidays],
        columns=["ds", "holiday"],
    )
    holidays_df = holidays_df.explode("holiday")
    holidays_df.reset_index(inplace=True, drop=True)
    holidays_df["ds"] = pd.to_datetime(holidays_df["ds"])

    return holidays_df


In [39]:
make_holidays_df(country="CN", year_list=np.arange(1995, 2045))

,ds,holiday
0,1995-01-01,New Year's Day
1,1995-01-31,Chinese New Year (Spring Festival)
2,1995-02-01,Chinese New Year (Spring Festival)
3,1995-02-02,Chinese New Year (Spring Festival)
4,1995-05-01,Labor Day
...,...,...
877,2044-02-02,Chinese New Year (Spring Festival) (observed)
878,2044-02-03,Chinese New Year (Spring Festival) (observed)
879,2044-05-03,Labor Day (observed)
880,2044-10-04,National Day (observed)
